In [25]:
#### Setting up Spark for colab space, code provided by Professor Othman

import os
#Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Set JAVA_HOME path variable in Linux
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version


#Install Spark
#download Spark file
!wget -q http://apache.osuosl.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
#extract the file
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
#Set SPARK-HOME path variable in Linux
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"


#install findspark package
!pip install -q findspark

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


In [26]:
import findspark
findspark.init()

In [27]:
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
conf = SparkConf().setAppName("lecture10").setMaster("local[*]")
sc=SparkContext(conf = conf)
spark = SparkSession(sparkContext=sc)
sqlContext = SQLContext(sc)

In [28]:
spark

In [29]:
# hardcoded variables telling our program which states, years, and tables are needed from NIBRS dataa

state_list = ["alabama", "arizona"]
state_abbr = ["AL", "AZ"]
csv = ["nibrs_offense", "nibrs_circumstances", "nibrs_victim_circumstances", "nibrs_victim"]
firstyear = 1991
lastyear = 2018

In [30]:
import pandas as pd

df_all = []   # empty list of all read in tables
url_base="https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/"

for j in range(0, len(csv)):
    df_list = []
    for state_i in range(0,len(state_list)):
        for year in range(firstyear,lastyear+1):
            url = url_base+state_list[state_i]+'/'+state_abbr[state_i]+'-'+str(year)
            url_c = url + '/' + csv[j] + '.csv'
            #print("Reading " + url_c)
            #print(url_c)
            try:
                temp_df = pd.read_csv(url_c, error_bad_lines=False)
            except:
                url_c = url + '/' + csv[j].upper() + '.csv'
                try:
                    temp_df = pd.read_csv(url_c, error_bad_lines=False)
                    temp_df.columns = map(str.lower, temp_df.columns)
                except:
                    #break
                    print(url_c + " did not exist")
            
            temp_df['state'] = state_abbr[state_i]
            df_list.append(temp_df)
            #print(df_list)
            #break
    df_all.append(pd.concat(df_list))
        
#AL2018_OFFENSE=pd.read_csv(url,  error_bad_lines=False)
#AL2018_OFFENSE

https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1993/NIBRS_OFFENSE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1994/NIBRS_OFFENSE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1995/NIBRS_OFFENSE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1996/NIBRS_OFFENSE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1997/NIBRS_OFFENSE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1998/NIBRS_OFFENSE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1999/NIBRS_OFFENSE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-2000/NIBRS_OFFENSE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/C

In [31]:
from pyspark.sql.types import *

incident_schema = StructType([ StructField("offense_id", IntegerType(), True)\

                       ,StructField("incident_id", IntegerType(), True)\

                       ,StructField("offense_type_id", IntegerType(), True)\

                       ,StructField("attempt_complete_flag", StringType(), True)\

                       ,StructField("location_id", IntegerType(), True)\

                       ,StructField("num_premises_entered", DoubleType(), True)\

                       ,StructField("method_entry_code", StringType(), True)\

                       ,StructField("ff_line_number", DoubleType(), True)\

                       ,StructField("state", StringType(), True)\

                       ,StructField("data_year", DoubleType(), True)])

df = spark.createDataFrame(df_all[0],schema=incident_schema)

del df_all[0]

In [32]:
from pyspark.sql.types import *

circumstances_schema = StructType([ StructField("circumstances_id", IntegerType(), True)\

                       ,StructField("circumstances_type", StringType(), True)\

                       ,StructField("circumstances_code", IntegerType(), True)\

                       ,StructField("circumstances_name", StringType(), True)\

                       ,StructField("state", StringType(), True)])

circumstances_spark = spark.createDataFrame(df_all[0],schema=circumstances_schema)
del df_all[0]

In [33]:
circumstances_spark = circumstances_spark.drop("state")
circumstances_spark = circumstances_spark.drop_duplicates()
circumstances_spark.show()
circumstances_spark.count()

+----------------+------------------+------------------+--------------------+
|circumstances_id|circumstances_type|circumstances_code|  circumstances_name|
+----------------+------------------+------------------+--------------------+
|              13|                 N|                30|Child Playing Wit...|
|               7|                 A|                 7|       Mercy Killing|
|              15|                 N|                32|    Hunting Accident|
|              17|                 N|                34|Other Negligent K...|
|               9|                 A|                 9| Other Circumstances|
|               6|                 A|                 6|      Lovers Quarrel|
|              12|                 J|                21|Criminal Killed b...|
|              10|                 A|                10|Unknown Circumsta...|
|              16|                 N|                33|Other Negligent W...|
|               3|                 A|                 3|        

18

In [34]:
from pyspark.sql.types import *

victim_circumstances_schema = StructType([ StructField("victim_id", IntegerType(), True)\

                       ,StructField("circumstances_id", IntegerType(), True)\

                       ,StructField("justifiable_force_id", DoubleType(), True)\

                       ,StructField("state", StringType(), True)\

                       ,StructField("data_year", DoubleType(), True)])

victim_circumstances_spark = spark.createDataFrame(df_all[0],schema=victim_circumstances_schema)
del df_all[0]

In [35]:
victim_circumstances_spark = victim_circumstances_spark.drop("state")
victim_circumstances_spark = victim_circumstances_spark.drop_duplicates()
victim_circumstances_spark.show()
victim_circumstances_spark.count()

+---------+----------------+--------------------+---------+
|victim_id|circumstances_id|justifiable_force_id|data_year|
+---------+----------------+--------------------+---------+
|    15240|              10|                 NaN|      NaN|
|    15241|              10|                 NaN|      NaN|
|    12878|              10|                 NaN|      NaN|
|    18894|              10|                 NaN|      NaN|
|    21502|              10|                 NaN|      NaN|
|    20788|              10|                 NaN|      NaN|
|    19260|              10|                 NaN|      NaN|
|     3259|              10|                 NaN|      NaN|
|     6769|              10|                 NaN|      NaN|
|    13420|              10|                 NaN|      NaN|
|    14327|              10|                 NaN|      NaN|
|    50871|              10|                 NaN|      NaN|
|    55243|              10|                 NaN|      NaN|
|    54515|              10|            

47260

In [36]:
victim_df = df_all[0].drop(columns=['victim_seq_num', 'victim_type_id', 'assignment_type_id','activity_type_id','outside_agency_id', \
                        'age_id', 'age_num', 'sex_code', 'resident_status_code', 'agency_data_year', 'ff_line_number', \
                        'age_range_low_num', 'age_range_high_num', 'data_year'])

In [37]:
victim_schema = StructType([ StructField("victim_id", IntegerType(), True)\

                       ,StructField("incident_id", IntegerType(), True)\

                       #,StructField("victim_seq_num", IntegerType(), True)\

                       #,StructField("victim_type_id", IntegerType(), True)\

                       #,StructField("assignment_type_id", DoubleType(), True)\

                       #,StructField("activity_type_id", DoubleType(), True)\

                       #,StructField("outside_agency_id", DoubleType(), True)\

                       #,StructField("age_id", DoubleType(), True)\

                       #,StructField("age_num", DoubleType(), True)\

                       #,StructField("sex_code", StringType(), True)\
                       
                       ,StructField("race_id", DoubleType(), True)\
                       
                       ,StructField("ethnicity_id", DoubleType(), True)\
                       
                       #,StructField("resident_status_code", StringType(), True)\
                       
                       #,StructField("agency_data_year", DoubleType(), True)\
                       
                       #,StructField("ff_line_number", DoubleType(), True)\
                       
                       #,StructField("age_range_low_num", DoubleType(), True)\
                       
                       #,StructField("age_range_high_num", DoubleType(), True)\
                       
                       ,StructField("state", StringType(), True)\
                       
                       #,StructField("data_year", DoubleType(), True)\
                       
                       ])

victim_spark = spark.createDataFrame(victim_df,schema=victim_schema)

In [38]:
victim_spark.show()


+---------+-----------+-------+------------+-----+
|victim_id|incident_id|race_id|ethnicity_id|state|
+---------+-----------+-------+------------+-----+
|   130114|     130114|    1.0|         3.0|   AL|
|   129982|     129982|    0.0|         3.0|   AL|
|   129925|     129925|    1.0|         3.0|   AL|
|   129924|     129924|    2.0|         3.0|   AL|
|   129923|     129923|    NaN|         NaN|   AL|
|   129922|     129922|    0.0|         3.0|   AL|
|   129921|     129921|    1.0|         3.0|   AL|
|   129920|     129920|    0.0|         3.0|   AL|
|   129918|     129918|    0.0|         3.0|   AL|
|   129915|     129915|    1.0|         3.0|   AL|
|   129914|     129914|    0.0|         3.0|   AL|
|   129913|     129913|    1.0|         3.0|   AL|
|   129912|     129912|    0.0|         3.0|   AL|
|   129911|     129911|    0.0|         3.0|   AL|
|   129910|     129910|    NaN|         NaN|   AL|
|   129908|     129908|    NaN|         NaN|   AL|
|   129907|     129907|    NaN|